In [ ]:
import pandas as pd
import numpy as np
import json
import pymc as pm
import arviz as az
az.style.use("arviz-doc")

from sensiblesleep.plot_functions import plot_screen_events, plot_posterior_sleep_wake, plot_logp, plot_DIC, plot_sleep_duration
from sensiblesleep.model_functions import is_asleep, create_time_bins, calculate_DIC
from sensiblesleep.utils import get_time_from_bin, extract_user_data

In [ ]:
CONFIG = json.load(open("sensiblesleep/config.json"))
MODELCONFIG = CONFIG["model"]
SAMPLECONFIG = CONFIG["sampling"]

# Synthetic data generation

In [ ]:
# Import the generate_synthetic_data function
from sensiblesleep.data_generation import generate_synthetic_data

# Generate the synthetic data
df = generate_synthetic_data()

# Display the first few rows
df.head()

# Extract user data

In [ ]:
n_days, hours, observed_event_counts = extract_user_data(df, user='user_001')
n_bins, total_bins, time_bins = create_time_bins(n_days=n_days)

# Plot the screen events
plot_screen_events(observed_event_counts, time_bins, hours)

# Models

## Pooled-Pooled Model

In [ ]:
# Build the Pooled-Pooled Model
with pm.Model() as pooled_pooled_model:
    
    # Priors for sleep and wake times (uniform between 0 and 96 bins)
    tsleep = pm.DiscreteUniform('tsleep', lower=0, upper=n_bins)
    tawake = pm.DiscreteUniform('tawake', lower=0, upper=n_bins)

    # Shared rate for awake periods
    lambda_awake = pm.Gamma('lambda_awake', alpha=MODELCONFIG['alpha_awake'], beta=MODELCONFIG['beta_awake'])

    # Very low rate for sleep periods (prior belief)
    lambda_sleep = pm.Exponential('lambda_sleep', lam=MODELCONFIG['lambda_sleep'])

    # Create the time bins (reshaped to match broadcasting requirements)
    # Expand tsleep and tawake to repeat each day’s value across 96 bins

    tsleep_expanded = np.repeat(tsleep, total_bins).reshape((total_bins, 1))
    tawake_expanded = np.repeat(tawake, total_bins).reshape((total_bins, 1))

    # Calculate the sleep indicator using the expanded tsleep and tawake
    sleep_indicator = is_asleep(time_bins, tsleep_expanded, tawake_expanded, n_bins)

    # Expected event rate
    event_rate = sleep_indicator * lambda_sleep + (1 - sleep_indicator) * lambda_awake
    
    # Likelihood
    likelihood = pm.Poisson('events', mu=event_rate, observed=observed_event_counts.reshape(-1, 1))

    # Sampling steps
    pooled_pooled_trace = pm.sample(
        draws=SAMPLECONFIG['n_samples'],
        tune=SAMPLECONFIG['tune'],
        cores=SAMPLECONFIG['cores'],
        target_accept=SAMPLECONFIG['target_accept'],
        compute_convergence_checks=SAMPLECONFIG['compute_convergence_checks'],
        random_seed=SAMPLECONFIG['random_seed']
    )
    
    
    #pm.sample(SAMPLECONFIG['n_samples'], chains=SAMPLECONFIG['n_chains'],tune=SAMPLECONFIG['tune'], cores=SAMPLECONFIG['cores'], target_accept=SAMPLECONFIG['target_accept'], compute_convergence_checks=SAMPLECONFIG['compute_convergence_checks'], random_seed=SAMPLECONFIG['random_seed'])
    pooled_pooled_pp = pm.sample_posterior_predictive(pooled_pooled_trace, progressbar=SAMPLECONFIG['progressbar'])
    likelihood_pooled_pooled = pm.compute_log_likelihood(pooled_pooled_trace)
    
logp_pooled_pooled = pooled_pooled_trace['sample_stats']['lp']

In [ ]:
az.summary(pooled_pooled_trace, var_names=['tsleep', 'tawake', 'lambda_sleep', 'lambda_awake'])
